In [3]:
import pandas
import numpy as np
pandas.__version__

'2.1.1'

## Read In Data

In [4]:
# read data from our csv
df = pandas.read_csv('2019-Present.csv')

In [5]:
df.head()

,Establishment ID,Inspection Date,Establishment Name,Establishment Address,Town,Violation Code,Violation Description,Violation Occurences
0,35-046524,1/8/2019,565,565 E 2100,Salt Lake City,4.3.31,In-Use Utensils - Between-Use Storage.,1.0
1,35-046524,1/8/2019,565,565 E 2100,Salt Lake City,4.3.56,Thawing,1.0
2,35-046524,1/8/2019,565,565 E 2100,Salt Lake City,4.4.64,Cutting Surfaces,1.0
3,35-046524,1/8/2019,565,565 E 2100,Salt Lake City,4.4.76*,Manual and Mechanical Warewashing Equipment-Ch...,1.0
4,35-046524,1/8/2019,565,565 E 2100,Salt Lake City,4.4.82*,"Equipment, Food-Contact Surfaces, Nonfood-Cont...",1.0


In [6]:
# Fill in NaN values with "N/A"
df = df.fillna("N/A")

## Add Columns to Identify Critical 1 and Critical 2 Violations

In [7]:
# Get the last character in Violation Code string, place into new column Critical Violation 1
# ref: https://stackoverflow.com/questions/52850192/python-extract-last-digit-of-a-string-from-a-pandas-column
df["Critical Violation 1"] = df["Violation Code"].str.strip().str[-1]
# If Critical Violation 1 contains "*", True, otherwise false
# ref: https://stackoverflow.com/questions/31511997/pandas-dataframe-replace-all-values-in-a-column-based-on-condition
df.loc[df["Critical Violation 1"] == "*", 'Critical Violation 1'] = True
df.loc[df["Critical Violation 1"] != True, 'Critical Violation 1'] = False

# Get the last two characters in Violation Code string, place into new column Critical Violation 2
df["Critical Violation 2"] = df["Violation Code"].str[-2:]
# If Critical Violation 2 contains "**", True, otherwise false
df.loc[df["Critical Violation 2"] == "**", 'Critical Violation 2'] = True
df.loc[df["Critical Violation 2"] != True, 'Critical Violation 2'] = False

# If both critical violation 1 and critical violation 2 are true, then it's just a critical 2 violation
df.loc[df['Critical Violation 2'] == True, "Critical Violation 1"] = False

In [8]:
# See the results
df.head()

,Establishment ID,Inspection Date,Establishment Name,Establishment Address,Town,Violation Code,Violation Description,Violation Occurences,Critical Violation 1,Critical Violation 2
0,35-046524,1/8/2019,565,565 E 2100,Salt Lake City,4.3.31,In-Use Utensils - Between-Use Storage.,1.0,False,False
1,35-046524,1/8/2019,565,565 E 2100,Salt Lake City,4.3.56,Thawing,1.0,False,False
2,35-046524,1/8/2019,565,565 E 2100,Salt Lake City,4.4.64,Cutting Surfaces,1.0,False,False
3,35-046524,1/8/2019,565,565 E 2100,Salt Lake City,4.4.76*,Manual and Mechanical Warewashing Equipment-Ch...,1.0,True,False
4,35-046524,1/8/2019,565,565 E 2100,Salt Lake City,4.4.82*,"Equipment, Food-Contact Surfaces, Nonfood-Cont...",1.0,True,False


## Remove Numerical Identifiers from Restaurant Chains

In [9]:
# Split names that have "#" on "#" and keep only the first string
# ref: https://stackoverflow.com/questions/40705480/python-pandas-remove-everything-after-a-delimiter-in-a-string
df["Establishment Name"] = df["Establishment Name"].str.split('#').str[0]

In [10]:
# see the results, Subway for example
df.loc[df['Establishment Name'] == "SUBWAY"].head()

,Establishment ID,Inspection Date,Establishment Name,Establishment Address,Town,Violation Code,Violation Description,Violation Occurences,Critical Violation 1,Critical Violation 2
133041,35-0015876,11/5/2019,SUBWAY,1000 S MAIN,Salt Lake City,4.4.62,Fixed Equipment-Elevation or Sealing,1.0,False,False
133042,35-0015876,11/5/2019,SUBWAY,1000 S MAIN,Salt Lake City,4.5.26**,Using a Handwashing Sink.**,1.0,False,True
133043,35-0015876,11/5/2019,SUBWAY,1000 S MAIN,Salt Lake City,4.6.49,Using Dressing Rooms and Lockers,1.0,False,False
133044,35-0015876,11/5/2019,SUBWAY,1000 S MAIN,Salt Lake City,4.7.16*,Restriction and Storage*,1.0,True,False
133045,35-0014316,8/9/2019,SUBWAY,11327 S JORDAN,Sandy,4.6.40,Repairing-Physical Facilities,1.0,False,False


## Remove Non-Public Establishments

In [13]:
# define what strings to remove
to_remove = "SCHOOL|CENTER|CHILD|LIVING|CORRECTION|ARAMARK|HEALTH|CARE|JAIL|KIDS|ELEMENTARY|DETENTION|FOOD BANK|RECOVERY|CAMP"
df = df[df["Establishment Name"].str.contains(to_remove)==False]

In [14]:
# see the results, for example, "ABOVE & BEYOND CHILDCARE" is gone
df.loc[df['Establishment Name'] == "ABOVE & BEYOND CHILDCARE"]

,Establishment ID,Inspection Date,Establishment Name,Establishment Address,Town,Violation Code,Violation Description,Violation Occurences,Critical Violation 1,Critical Violation 2


## Add a Column Specifying 7.4.4 Emergency Enforcement Violations

In [15]:
# # Create a column called Emergency Enforcement Violation, populate with content of Violation Code
df["Emergency Enforcement Violation"] = df["Violation Code"]
# If Emergency Enforcement Violation contains 7.4.4 True, otherwise false
df.loc[df["Violation Code"].str.contains("7.4.4"), "Emergency Enforcement Violation"] = True
df.loc[df["Emergency Enforcement Violation"] != True, "Emergency Enforcement Violation"] = False

In [16]:
# see the results
df.loc[df['Violation Code'].str.contains("7.4.4")].head()

,Establishment ID,Inspection Date,Establishment Name,Establishment Address,Town,Violation Code,Violation Description,Violation Occurences,Critical Violation 1,Critical Violation 2,Emergency Enforcement Violation
1951,35-045554,7/8/2020,ALPINE MEADOW REAHABILITATION AND NURSING,2520 S REDWOOD,West Valley City,7.4.4 (ii),Summary Permit Suspension - Imminent Health Ha...,1.0,False,False,True
2820,35-041669,2/14/2019,AMKHA MISKY PERUVIAN SPECIALTIES,215 S 800,Salt Lake City,7.4.4 (ii),Summary Permit Suspension - Imminent Health Ha...,1.0,False,False,True
3198,35-060708,5/4/2023,ANTOJITOS EL COLOCHO,4219 W 3500,West Valley City,7.4.4 (ii),Summary Permit Suspension - Imminent Health Ha...,1.0,False,False,True
3737,35-042173,6/4/2022,APOLLO BURGERS,940 W 1700,Salt Lake City,7.4.4 (ii),Summary Permit Suspension - Imminent Health Ha...,1.0,False,False,True
3955,35-0012368,1/19/2023,APPLE SPICE JUNCTION,6520 S 900,Murray,7.4.4 (ii),Summary Permit Suspension - Imminent Health Ha...,1.0,False,False,True


## Remove Rows Containing Non-Violations or Undefined Violations

In [17]:
# define what strings to remove
to_remove = "R392-100: 5-202.11*|R392-510UICAA"
df = df[df["Violation Code"].str.contains(to_remove)==False]

In [18]:
# remove rows where violation code begins with 0
df = df[df["Violation Code"].str.startswith("0")==False]

In [19]:
# see the results, no violations that begin with 0
df.loc[df['Violation Code'].str.startswith("0")]

,Establishment ID,Inspection Date,Establishment Name,Establishment Address,Town,Violation Code,Violation Description,Violation Occurences,Critical Violation 1,Critical Violation 2,Emergency Enforcement Violation


In [20]:
# no violations using codes above
df.loc[df['Violation Code'].str.contains("R392")]

,Establishment ID,Inspection Date,Establishment Name,Establishment Address,Town,Violation Code,Violation Description,Violation Occurences,Critical Violation 1,Critical Violation 2,Emergency Enforcement Violation


## Add a Column Specifying 7.4.2 Variances

In [21]:
# # Create a column called Emergency Enforcement Violation, populate with content of Violation Code
df["Variance"] = df["Violation Code"]
# If Emergency Enforcement Violation contains 7.4.4 True, otherwise false
df.loc[df["Violation Code"].str.contains("7.4.2"), "Variance"] = True
df.loc[df["Variance"] != True, "Variance"] = False

In [22]:
# see the results, variances are now labeled
df.loc[df['Violation Code'].str.contains("7.4.2")].head()

,Establishment ID,Inspection Date,Establishment Name,Establishment Address,Town,Violation Code,Violation Description,Violation Occurences,Critical Violation 1,Critical Violation 2,Emergency Enforcement Violation,Variance
19772,35-051497,6/11/2019,CAFE RIO,3585 S 8400,Magna,7.4.2 (iii),Conformance With Approved Procedures,1.0,False,False,False,True
53068,35-063219,7/8/2021,GOURMANDISE THE BAKERY DOG PATIO,250 S 300,Salt Lake City,7.4.2 (ii),Documentation of Proposed Variance and Justifi...,1.0,False,False,False,True
55580,35-068491,1/11/2023,HALL PASS - DOG PATIO,153 S RIO GRANDE,Salt Lake City,7.4.2 (iii),Conformance With Approved Procedures,1.0,False,False,False,True
55852,35-055805,10/17/2019,HANAPOKE SUGARHOUSE,675 E 2100,Salt Lake City,7.4.2 (ii),Documentation of Proposed Variance and Justifi...,1.0,False,False,False,True
67613,35-065809,1/27/2023,JUICE DRIP,807 E PIONEER,Draper,7.4.2 (i),Variances Modifications and Waivers,1.0,False,False,False,True


## Remove Astrisks from Violation Codes

In [26]:
# ref for removing characters:
# https://saturncloud.io/blog/what-is-the-best-way-to-remove-characters-from-a-string-in-pandas/
# https://pandas.pydata.org/docs/reference/api/pandas.Series.str.replace.html
df['Violation Code'] = df['Violation Code'].str.replace('*', '', regex=False)

In [27]:
# see results, astrisks are removed
df.head()

,Establishment ID,Inspection Date,Establishment Name,Establishment Address,Town,Violation Code,Violation Description,Violation Occurences,Critical Violation 1,Critical Violation 2,Emergency Enforcement Violation,Variance
0,35-046524,1/8/2019,565,565 E 2100,Salt Lake City,4.3.31,In-Use Utensils - Between-Use Storage.,1.0,False,False,False,False
1,35-046524,1/8/2019,565,565 E 2100,Salt Lake City,4.3.56,Thawing,1.0,False,False,False,False
2,35-046524,1/8/2019,565,565 E 2100,Salt Lake City,4.4.64,Cutting Surfaces,1.0,False,False,False,False
3,35-046524,1/8/2019,565,565 E 2100,Salt Lake City,4.4.76,Manual and Mechanical Warewashing Equipment-Ch...,1.0,True,False,False,False
4,35-046524,1/8/2019,565,565 E 2100,Salt Lake City,4.4.82,"Equipment, Food-Contact Surfaces, Nonfood-Cont...",1.0,True,False,False,False


## Make strings lowercase

In [41]:
def capitalize(input):
    words = input.split()
    for i in range(len(words)):
        w = words[i].lower()
        if len(w) == 1 and w != "i":
            w = w.upper()
        elif len(w) >= 4 or i == 0:
            w = w[0].upper() + w[1:]
        words[i] = w
    return " ".join(words)

df['Establishment Name'] = df['Establishment Name'].apply(capitalize)
df['Establishment Address'] = df['Establishment Address'].apply(capitalize)
df['Town'] = df['Town'].apply(capitalize)

## Save Data

In [42]:
# Save as json file
# ref: https://www.geeksforgeeks.org/exporting-pandas-dataframe-to-json-file/
df.to_json('data.json', orient = 'split', compression = 'infer', index = 'true')

In [43]:
# Save as csv
# ref: https://www.geeksforgeeks.org/saving-a-pandas-dataframe-as-a-csv/
df.to_csv('data.csv')